In [ ]:
!pip install semantic-text-similarity

In [ ]:
from semantic_text_similarity.models import WebBertSimilarity
from semantic_text_similarity.models import ClinicalBertSimilarity

web_model = WebBertSimilarity(device='cpu', batch_size=10) #defaults to GPU prediction

# clinical_model = ClinicalBertSimilarity(device='cuda', batch_size=10) #defaults to GPU prediction

In [ ]:
import numpy as np
import pdb
import json
class GFG: 
  def __init__(self,graph): 
    self.graph = graph 
    self.ppl = len(graph) 
    self.jobs = len(graph[0]) 

  def bpm(self, u, matchR, seen): 
    for v in range(self.jobs):  
      if self.graph[u][v] and seen[v] == False: 
        seen[v] = True
        if matchR[v] == -1 or self.bpm(matchR[v], matchR, seen): 
          matchR[v] = u 
          return True
    return False
  
  # Returns maximum number of matching 
  def maxBPM(self):
    matchR = [-1] * self.jobs 

    result = 0
    for i in range(self.ppl):
      seen = [False] * self.jobs
      if self.bpm(i, matchR, seen): 
        result += 1
    return result, matchR

def my_lcs(string, sub):
  if(len(string)<= len(sub)):
    sub, string = string, sub

    lengths = [[0 for i in range(0,len(sub)+1)] for j in range(0,len(string)+1)]

    for j in range(1,len(sub)+1):
      for i in range(1,len(string)+1):
        if (string[i-1] == sub[j-1]):
          lengths[i][j] = lengths[i-1][j-1] + 1
        else:
          lengths[i][j] = max(lengths[i-1][j] , lengths[i][j-1])

    return lengths[len(string)][len(sub)]

class Rouge():
  def __init__(self):
    self.beta = 1.2

  def calc_score(self, candidate, refs):
    assert(len(candidate)==1)	
    assert(len(refs)>0)         
    prec = []
    rec = []

  # split into tokens
    token_c = candidate[0].split(" ")
    	
    for reference in refs:
      # split into tokens
      token_r = reference.split(" ")
      # compute the longest common subsequence
      lcs = my_lcs(token_r, token_c)
      if (lcs == None):
        prec.append(0)
        rec.append(0)
      else:
        prec.append(lcs/float(len(token_c)))
        rec.append(lcs/float(len(token_r)))

      prec_max = max(prec)
      rec_max = max(rec)

      if (prec_max!=0 and rec_max !=0):
        score = ((1 + self.beta**2)*prec_max*rec_max)/float(rec_max + self.beta**2*prec_max)
      else:
        score = 0.0
      return score

  def compute_score(self, refs, test):
    score = []
    for i in range(len(refs)):
      hypo = test[i]
      ref = refs[i]
      if (hypo == " " or hypo == ""):
        score.append(0)
      else:
        score.append(self.calc_score([hypo], [ref]))
    
    average_score = np.mean(np.array(score))
    return average_score, np.array(score)

  def method(self):
    return "Rouge"

For evaluation of property generation models XGP and XGP-W

In [ ]:
f = open('./GPT_Multiple_Output/gpt2_raw_output.json')
data = json.load(f)
name = 'gpt2_raw_output'

In [ ]:
import json

sts_predictions_array = []
sts_predictions_array2 = []
positive_indices = []
negative_indices = []
sts_recall = 0.0
sts_precision = 0.0
sts_fscore = 0.0
count = 0
sts_threshold = 3

counter = []
for k in range(len(data["Input"])):
  if (k % 500 == 0):
    print(k)
  l1 = data["Gold"][k]
  l2 = data["Output"][k]
  if data["Correctness"][k]:
    positive_indices.append(k)
    # l2 = data["Output"][k]
  else:
    negative_indices.append(k)
    # l2 = [data["Output"][k][0]]
  bipartite_graph = np.zeros((len(l1), len(l2)))
  bipartite_graph_double = np.zeros((len(l1), len(l2)))
  
  for i in range(len(l1)):
    for j in range(len(l2)):
      sts_score = web_model.predict([(l1[i], l2[j])])[0]
      bipartite_graph_double[i][j] = sts_score
      if (sts_score >= sts_threshold):
        bipartite_graph[i][j] = 1
      else:
        bipartite_graph[i][j] = 0

  g = GFG(bipartite_graph_double) 
  number, division_list = g.maxBPM()

  # property i will be matched with division_list[i]  change this comment
  score0 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if(division_list[k] == i):
        j = k
        break

    if (j != -1):
      sts_score = bipartite_graph_double[i][j]
      score0 += sts_score
      count += 1
      counter.append(count)
      sts_predictions_array2.append(sts_score)
      
    else:
      count += 1
      sts_predictions_array2.append(0)

  sts_predictions_array.append(score0/len(l1))

  g = GFG(bipartite_graph) 
  number, division_list = g.maxBPM()

  # property i will be matched with division_list[i]  change this comment
  score_recall0 = 0
  score_precision0 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if(division_list[k] == i):
        j = k
        break

    if (j != -1):
      score_recall0 += 1
      score_precision0 += 1
      count += 1
    else:
      count += 1
  sts_recall += score_recall0/len(l1)
  sts_precision += score_precision0/len(l2)
  a0 = score_recall0/len(l1)
  b0 = score_precision0/len(l2)
  if (a0+b0 != 0):
    sts_fscore += 2*a0*b0/(a0+b0)

# print(count)
# print(len(counter))
x = len(data["Input"])
print("STS Score==============")
print(sts_recall/x)
print(sts_precision/x)
print(sts_fscore/x)
# print(np.average(sts_predictions_array)/5)
# predictions_positive = [sts_predictions_array[i] for i in positive_indices]
# predictions_negative = [sts_predictions_array[i] for i in negative_indices]
# print(np.average(predictions_positive)/5)
# print(np.average(predictions_negative)/5)

In [ ]:
spice_thresholder = []
cider_refs_thresholder = []
cider_test_thresholder = []
count = 0
for k in range(len(data["Input"])):
  # if (k % 500 == 0):
  #   print(k)
  l1 = data["Gold"][k]
  l2 = data["Output"][k]
  # correctness = data["Correctness"][k]
  # if not correctness:
  #   l2 = [data["Output"][k][0]]
  # else:
  #   l2 = data["Output"][k]
  for i in range(len(l1)):
    for j in range(len(l2)):
      struct = {
          "image_id": count,
          "caption": l2[j]
      }
      cider_test_thresholder.append(struct)
      struct = {
          "image_id": count,
          "caption": l1[i]
      }
      cider_refs_thresholder.append(struct)
      struct = {
          "image_id": count,
          "test": l2[j],
          "refs": [l1[i]]
      }
      spice_thresholder.append(struct)
      count += 1

In [ ]:
%cd cider/

In [ ]:
with open('./data/cider_' + name + '_refs.json', 'w') as outfile:
    json.dump(cider_refs_thresholder, outfile)
with open('./data/cider_' + name + '_test.json', 'w') as outfile:
    json.dump(cider_test_thresholder, outfile)
with open('../spice/spice_' + name + '.json', 'w') as outfile:
    json.dump(spice_thresholder, outfile)

params = {
  "pathToData" : "data/",
	"refName" : 'cider_' + name + '_refs.json',
	"candName" : 'cider_' + name + '_test.json',
	"resultFile" : 'cider_' + name + '_results.json',
	"idf" : "coco-val-df"
}
with open('params.json', 'w') as outfile:
    json.dump(params, outfile)

In [ ]:
!python2 cidereval.py

In [ ]:
file2 = open('./cider_' + name + '_results.json')
cider_output = json.load(file2)

In [ ]:
%cd ../meteor/
write_file = open(name + "_meteor_refs", "w")
for i in range(len(cider_refs_thresholder)):
  new_line = cider_refs_thresholder[i]['caption'].replace("\n", " ") + " \n"
  write_file.write(new_line)
write_file.close()

write_file2 = open(name + "_meteor_test", "w")
for i in range(len(cider_test_thresholder)):
  if (cider_test_thresholder[i]['caption'] == "" or cider_test_thresholder[i]['caption'] == " "):
    new_line = "empty \n"
  else:
    new_line = cider_test_thresholder[i]['caption'].replace("\n", " ") + " \n"
  write_file2.write(new_line)
write_file2.close()

In [ ]:
meteor_scores = !java -Xmx2G -jar meteor-1.5.jar ./gpt2_raw_output_meteor_test ./gpt2_raw_output_meteor_refs -l en -norm -a data/paraphrase-en.gz -q
meteor_scores = [float(meteor_scores[i]) for i in range(len(meteor_scores))]
meteor_scores[-1]

In [ ]:
%cd ../spice/

In [ ]:
#in spice directory
!java -Xmx8G -jar spice-1.0.jar spice_gpt2_raw_output.json -cache ./cache -out spice_gpt2_raw_output_output.json

In [ ]:
file2 = open('./spice_' + name + '_output.json')
spice_output = json.load(file2)
len(spice_output)

In [ ]:
rouge_test =  [cider_test_thresholder[i]['caption'] for i in range(len(cider_test_thresholder))]
rouge_refs =  [cider_refs_thresholder[i]['caption'] for i in range(len(cider_refs_thresholder))]
r = Rouge()
rouge_scores = r.compute_score(rouge_refs, rouge_test)
rouge_scores[0]

In [ ]:
import json

spice_recall = 0.0
spice_precision = 0.0
spice_fscore = 0.0
cider_recall = 0.0
cider_precision = 0.0
cider_fscore = 0.0
meteor_recall = 0.0
meteor_precision = 0.0
meteor_fscore = 0.0
rouge_recall = 0.0
rouge_precision = 0.0
rouge_fscore = 0.0
count1 = 0
count = 0
spice_threshold = 0.4
cider_threshold = 3
meteor_threshold = 0.3
rouge_threshold = 0.3
counter = []
for k in range(len(data["Input"])):
  if (k % 500 == 0):
    print(k)
  l1 = data["Gold"][k]
  l2 = data["Output"][k]
  
  bipartite_graph = np.zeros((len(l1), len(l2)))
  bipartite_graph_double_spice = np.zeros((len(l1), len(l2)))
  bipartite_graph_double_meteor = np.zeros((len(l1), len(l2)))
  bipartite_graph_double_rouge = np.zeros((len(l1), len(l2)))
  
  for i in range(len(l1)):
    for j in range(len(l2)):
      cider_score = cider_output['CIDEr'][count1]
      meteor_score = meteor_scores[count1]
      rouge_score = rouge_scores[1][count1]
      spice_score = spice_output[count1]['scores']['All']['f']
      count1 += 1
      if (spice_score >= spice_threshold):
        bipartite_graph_double_spice[i][j] = 1
      else:
        bipartite_graph_double_spice[i][j] = 0
      if (cider_score >= cider_threshold):
        bipartite_graph[i][j] = 1
      else:
        bipartite_graph[i][j] = 0
      if (meteor_score >= meteor_threshold):
        bipartite_graph_double_meteor[i][j] = 1
      else:
        bipartite_graph_double_meteor[i][j] = 0
      if (rouge_score >= rouge_threshold):
        bipartite_graph_double_rouge[i][j] = 1
      else:
        bipartite_graph_double_rouge[i][j] = 0

  g = GFG(bipartite_graph_double_spice)
  number, division_list = g.maxBPM()
  
  score_recall1 = 0
  score_precision1 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if (division_list[k] == i):
        j = k
        break
    
    if (j != -1):
      score_recall1 += 1
      score_precision1 += 1
      count += 1
    else:
      count += 1

  spice_recall += score_recall1/len(l1)
  spice_precision += score_precision1/len(l2)
  a1 = score_recall1/len(l1)
  b1 = score_precision1/len(l2)
  if (a1+b1 != 0):
    spice_fscore += 2*a1*b1/(a1+b1)

  g = GFG(bipartite_graph)
  number, division_list = g.maxBPM()
  
  score_recall2 = 0
  score_precision2 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if (division_list[k] == i):
        j = k
        break
    
    if (j != -1):
      score_recall2 += 1
      score_precision2 += 1
      count += 1 
    else:
      count += 1

  cider_recall += score_recall2/len(l1)
  cider_precision += score_precision2/len(l2)
  a2 = score_recall2/len(l1)
  b2 = score_precision2/len(l2)
  if (a2+b2 != 0):
    cider_fscore += 2*a2*b2/(a2+b2)

  g = GFG(bipartite_graph_double_meteor) 
  number, division_list = g.maxBPM()
  
  score_recall3 = 0
  score_precision3 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if (division_list[k] == i):
        j = k
        break
    if (j != -1):
      score_recall3 += 1
      score_precision3 += 1
      count += 1 
    else:
      count += 1
  meteor_recall += score_recall3/len(l1)
  meteor_precision += score_precision3/len(l2)
  a2 = score_recall3/len(l1)
  b2 = score_precision3/len(l2)
  if (a2+b2 != 0):
    meteor_fscore += 2*a2*b2/(a2+b2)

  g = GFG(bipartite_graph_double_rouge) 
  number, division_list = g.maxBPM()
  
  score_recall4 = 0
  score_precision4 = 0
  for i in range(len(l1)):
    j = -1
    for k in range(len(division_list)):
      if (division_list[k] == i):
        j = k
        break
    if (j != -1):
      score_recall4 += 1
      score_precision4 += 1
      count += 1 
    else:
      count += 1
  rouge_recall += score_recall4/len(l1)
  rouge_precision += score_precision4/len(l2)
  a2 = score_recall4/len(l1)
  b2 = score_precision4/len(l2)
  if (a2+b2 != 0):
    rouge_fscore += 2*a2*b2/(a2+b2)

x = len(data["Input"])
print("SPICE==============")
print(spice_recall/x)
print(spice_precision/x)
print(spice_fscore/x)
print("CIDEr==============")
print(cider_recall/x)
print(cider_precision/x)
print(cider_fscore/x)
print("METEOR==============")
print(meteor_recall/x)
print(meteor_precision/x)
print(meteor_fscore/x)
print("ROUGE==============")
print(rouge_recall/x)
print(rouge_precision/x)
print(rouge_fscore/x)

For evaluation of free-flow generation models XGF-I and XGF-II

In [ ]:
import json
f = open('../GPT_Multiple_Output/gpt2_raw_freeflow_output.json')
data = json.load(f)
name = "gpt2_raw_freeflow_output"

spice_thresholder = []
cider_refs_thresholder = []
cider_test_thresholder = []
count = 0
for k in range(len(data["input"])):
  # if (k % 500 == 0):
  #   print(k)
  l1 = data["gold"][k].replace("<EOS>","")
  l2 = data["output"][k]
  # l1 = gold[k]
  # l2 = prime[k]
  struct = {
      "image_id": count,
      "caption": l2
      }
  cider_test_thresholder.append(struct)
  struct = {
      "image_id": count,
      "caption": l1
  }
  cider_refs_thresholder.append(struct)
  struct = {
      "image_id": count,
      "test": l2,
      "refs": [l1]
  }
  spice_thresholder.append(struct)
  count += 1
  

In [ ]:
%cd ../cider

In [ ]:
with open('./data/cider_' + name + '_refs.json', 'w') as outfile:
    json.dump(cider_refs_thresholder, outfile)
with open('./data/cider_' + name + '_test.json', 'w') as outfile:
    json.dump(cider_test_thresholder, outfile)
with open('../spice/spice_' + name + '.json', 'w') as outfile:
    json.dump(spice_thresholder, outfile)

params = {
  "pathToData" : "data/",
	"refName" : 'cider_' + name + '_refs.json',
	"candName" : 'cider_' + name + '_test.json',
	"resultFile" : 'cider_' + name + '_results.json',
	"idf" : "coco-val-df"
}
with open('params.json', 'w') as outfile:
    json.dump(params, outfile)

In [ ]:
!python2 cidereval.py

In [ ]:
file2 = open('./cider_' + name + '_results.json')
cider_output = json.load(file2)

In [ ]:
%cd ../spice/
write_file = open(name + "_meteor_refs", "w")
for i in range(len(cider_refs_thresholder)):
  new_line = cider_refs_thresholder[i]['caption'].replace("\n", " ") + " \n"
  write_file.write(new_line)
write_file.close()

write_file2 = open(name + "_meteor_test", "w")
for i in range(len(cider_test_thresholder)):
  if (cider_test_thresholder[i]['caption'] == "" or cider_test_thresholder[i]['caption'] == " "):
    new_line = "empty \n"
  else:
    new_line = cider_test_thresholder[i]['caption'].replace("\n", " ") + " \n"
  write_file2.write(new_line)
write_file2.close()

In [ ]:
meteor_scores = !java -Xmx2G -jar meteor-1.5.jar ./gpt2_raw_freeflow_output_meteor_test ./gpt2_raw_freeflow_output_meteor_refs -l en -norm -a data/paraphrase-en.gz -q
meteor_scores = [float(meteor_scores[i]) for i in range(len(meteor_scores))]
meteor_scores[-1]

In [ ]:
#in spice directory
!java -Xmx8G -jar spice-1.0.jar spice_gpt2_raw_freeflow_output.json -cache ./cache2 -out spice_gpt2_raw_freeflow_output_output.json

In [ ]:
file2 = open('./spice_' + name + '_output.json')
spice_output = json.load(file2)
len(spice_output)

In [ ]:
rouge_test =  [cider_test_thresholder[i]['caption'] for i in range(len(cider_test_thresholder))]
rouge_refs =  [cider_refs_thresholder[i]['caption'] for i in range(len(cider_refs_thresholder))]
r = Rouge()
rouge_scores = r.compute_score(rouge_refs, rouge_test)
rouge_scores[0]

In [ ]:
import json
import numpy as np
spice_predictions_array = []
cider_predictions_array = []
meteor_predictions_array = []
rouge_predictions_array = []
counter = []
for k in range(len(spice_output)):
  if (k % 500 == 0):
    print(k)
  # l1 = data["Gold"][k]
  # l2 = data["Output"][k]
  # if data["Correctness"][k]:
  #   positive_indices.append(k)
  # else:
  #   negative_indices.append(k)
  spice_predictions_array.append(spice_output[k]['scores']['All']['f'])
  cider_predictions_array.append(cider_output["CIDEr"][k])
  meteor_predictions_array.append(meteor_scores[k])
  rouge_predictions_array.append(rouge_scores[1][k])

# print(count)
# print(len(counter))
# # x = len(data["q_text"])
print("SPICE==============")
print(np.average(spice_predictions_array))
print("CIDEr==============")
print(np.average(cider_predictions_array)/10)
print("METEOR==============")
print(np.average(meteor_predictions_array))
print("ROUGE==============")
print(np.average(rouge_predictions_array))

In [ ]:
sts_bert_output = [web_model.predict([(data['gold'][i], data['output'][i])])[0] for i in range(len(data['input']))]
print("STS-BERT===========")
print(np.average(sts_bert_output)/5)